# Rf 6 0 3_Multicpu
Likelihood and minimization: setting up a multi-core parallelized unbinned maximum likelihood fit



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:58 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooPolynomial.h"
#include "RooAddPdf.h"
#include "RooProdPdf.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create 3d pdf and data
 -------------------------------------------

Create observables

In [3]:
RooRealVar x("x", "x", -5, 5);
RooRealVar y("y", "y", -5, 5);
RooRealVar z("z", "z", -5, 5);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create signal pdf gauss(x)*gauss(y)*gauss(z)

In [4]:
RooGaussian gx("gx", "gx", x, RooConst(0), RooConst(1));
RooGaussian gy("gy", "gy", y, RooConst(0), RooConst(1));
RooGaussian gz("gz", "gz", z, RooConst(0), RooConst(1));
RooProdPdf sig("sig", "sig", RooArgSet(gx, gy, gz));

Create background pdf poly(x)*poly(y)*poly(z)

In [5]:
RooPolynomial px("px", "px", x, RooArgSet(RooConst(-0.1), RooConst(0.004)));
RooPolynomial py("py", "py", y, RooArgSet(RooConst(0.1), RooConst(-0.004)));
RooPolynomial pz("pz", "pz", z);
RooProdPdf bkg("bkg", "bkg", RooArgSet(px, py, pz));

Create composite pdf sig+bkg

In [6]:
RooRealVar fsig("fsig", "signal fraction", 0.1, 0., 1.);
RooAddPdf model("model", "model", RooArgList(sig, bkg), fsig);

Generate large dataset

In [7]:
RooDataSet *data = model.generate(RooArgSet(x, y, z), 200000);

Parallel fitting
 -------------------------------

In parallel mode the likelihood calculation is split in n pieces,
 that are calculated in parallel and added a posteriori before passing
 it back to MINUIT.

Use four processes and time results both in wall time and cpu time

In [8]:
model.fitTo(*data, NumCPU(4), Timer(kTRUE));

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 4 remote server process.
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 fsig         1.00000e-01  5.00000e-02    0.00000e+00  1.00000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SPEED AGAINST RELIABILITY
 **********
 **    6 **MIGRAD         500           1
 **********
 FIRST CALL TO USER FUNCTION AT NEW START POINT, WITH IFLAG=4.
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (sig,bkg)
[#1] INFO:Minization --  The following expressions have been identified as constant 

Parallel mc projections
 ----------------------------------------------

Construct signal, total likelihood projection on (y,z) observables and likelihood ratio

In [9]:
RooAbsPdf *sigyz = sig.createProjection(x);
RooAbsPdf *totyz = model.createProjection(x);
RooFormulaVar llratio_func("llratio", "log10(@0)-log10(@1)", RooArgList(*sigyz, *totyz));

Calculate likelihood ratio for each event, define subset of events with high signal likelihood

In [10]:
data->addColumn(llratio_func);
RooDataSet *dataSel = (RooDataSet *)data->reduce(Cut("llratio>0.7"));

Make plot frame and plot data

In [11]:
RooPlot *frame = x.frame(Title("Projection on X with LLratio(y,z)>0.7"), Bins(40));
dataSel->plotOn(frame);

Perform parallel projection using mc integration of pdf using given input dataset.
 In this mode the data-weighted average of the pdf is calculated by splitting the
 input dataset in N equal pieces and calculating in parallel the weighted average
 one each subset. The N results of those calculations are then weighted into the
 final result

Use four processes

In [12]:
model.plotOn(frame, ProjWData(*dataSel), NumCPU(4));

new TCanvas("rf603_multicpu", "rf603_multicpu", 600, 600);
gPad->SetLeftMargin(0.15);
frame->GetYaxis()->SetTitleOffset(1.6);
frame->Draw();

[#1] INFO:Plotting -- RooAbsReal::plotOn(model) plot on x averages using data variables (y,z)
[#1] INFO:Plotting -- RooAbsReal::plotOn(model) only the following components of the projection data will be used: (y,z)
[#1] INFO:Plotting -- RooDataWeightedAverage::ctor(modelDataWgtAvg) constructing data weighted average of function model_Norm[x] over 14497 data points of (y,z) with a total weight of 14497
[#1] INFO:Plotting -- RooDataWeightedAverage::ctor(modelDataWgtAvg) constructing data weighted average of function model_Norm[x] over 14497 data points of (y,z) with a total weight of 14497
[#1] INFO:Eval -- RooAbsTestStatistic::initMPMode: started 4 remote server process.
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (gy,gz,py,pz)
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (gy,gz,py,pz)
[#1] INFO:Minization --  The following expression

Draw all canvases 

In [13]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()